In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()
from process_data import save_pickle, load_pickle, load_task

In [2]:
train_data = load_task('./dataset/train-v1.1.json')
dev_data = load_task('./dataset/dev-v1.1.json')
save_pickle(train_data, 'pickle/train_data.pickle')
save_pickle(dev_data, 'pickle/dev_data.pickle')

vocab = set()
for context, _, q, answer in train_data+dev_data:
    vocab |= set(context + q + answer)
vocab = list(sorted(vocab))
w2i = dict((c, i) for i, c in enumerate(vocab, 0))
i2w = dict((i, c) for i, c in enumerate(vocab, 0))
save_pickle(vocab, 'pickle/vocab.pickle')
save_pickle(w2i, 'pickle/w2i.pickle')
save_pickle(i2w, 'pickle/i2w.pickle')
train_data = load_pickle('pickle/train_data.pickle')
vocab = load_pickle('pickle/vocab.pickle')
w2i = load_pickle('pickle/w2i.pickle')

vocab_size = len(vocab)
embd_size = 128
context_maxlen = max(map(len, (c for c, _, _, _ in train_data)))
question_maxlen = max(map(len, (q for _, _, q, _ in train_data)))
print('vocab size:', vocab_size)
print('embd size:', embd_size)
print('context_maxlen:', context_maxlen)
print('question_maxlen:', question_maxlen)

dataset version: 1.1
load_task: 0 / 442
load_task: 100 / 442
load_task: 200 / 442
load_task: 300 / 442
load_task: 400 / 442
dataset version: 1.1
load_task: 0 / 48
save pickle to pickle/train_data.pickle
save pickle to pickle/dev_data.pickle
save pickle to pickle/vocab.pickle
save pickle to pickle/w2i.pickle
save pickle to pickle/i2w.pickle
load pickle/train_data.pickle
load pickle/vocab.pickle
load pickle/w2i.pickle
vocab size: 186069
embd size: 128
context_maxlen: 766
question_maxlen: 60


In [9]:
sent1 = list('i have a cat')
sent2 = list('i had a aieu')
data = [sent1, sent2]
vocab = set(sent1+sent2)
vocab_size = len(vocab)
print(data)
print('vocab_size', vocab_size)
w2i = {w:i for i, w in enumerate(vocab)}
i2w = {i:w for i, w in enumerate(vocab)}

max_word_len = max([len(word) for word in data])
print('max_word_len', max_word_len)

[['i', ' ', 'h', 'a', 'v', 'e', ' ', 'a', ' ', 'c', 'a', 't'], ['i', ' ', 'h', 'a', 'd', ' ', 'a', ' ', 'a', 'i', 'e', 'u']]
vocab_size 10
max_word_len 12


In [22]:
class CharEmbedding(nn.Module):
    def __init__(self, vocab_size, embd_size, out_chs, filters):
        super(CharEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embd_size)
        # nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, ...
        self.conv = nn.ModuleList([nn.Conv2d(1, out_chs, (f[0], f[1])) for f in filters])
        self.dropout = nn.Dropout(.5)
        self.fc1 = nn.Linear(out_chs*len(filters), 1)
        
    def forward(self, x):
        bs = x.size(0)
        x = self.embedding(x) # (N, seq_len, embd_dim)
        print('embd', x.size())
        x = x.unsqueeze(1) # (N, Cin, W, embd_dim), insert Channnel-In dim
        print('unsq', x.size())
        print('conv[0](x)', self.conv[0](x).size())
        # Conv2d
        #    Input : (N,Cin, Hin, Win )
        #    Output: (N,Cout,Hout,Wout) 
        # squeeze(3) means 2D to 1D; (N,Cout,Hout,Wout) -> [(N,Cout,Hout==seq_len)] * len(filter_heights)
        x = [F.relu(conv(x)) for conv in self.conv]
        x = [xx.view((xx.size(0), xx.size(1), xx.size(2)*xx.size(3))) for xx in x]
        print('x[0]', x[0].size())
        # max_pool1d(input, kernel_size, ..
        # (N, Cout, seq_len) --(max_pool1d)--> (N, Cout, 1) --(squeeze(2))--> (N, Cout)
        # [(N, Cout)]  len(filter_heights)
        x = [F.max_pool1d(xx, xx.size(2)).squeeze(2) for xx in x]
        print('after maxpool', x[0].size())
        out = torch.cat(x, 1) # (N, Cout*len(filter_heights))
        print('out', out.size())
        return out

embd_size = 64
n_out_ch = 100
filters = [[1, 5]]
char_embd_net = CharEmbedding(vocab_size, embd_size, n_out_ch, filters)
print(char_embd_net)
char_var = Variable(torch.LongTensor([[w2i[w] for w in data[0]]]))
out = char_embd_net(char_var)

CharEmbedding (
  (embedding): Embedding(10, 64)
  (conv): ModuleList (
    (0): Conv2d(1, 100, kernel_size=(1, 5), stride=(1, 1))
  )
  (dropout): Dropout (p = 0.5)
  (fc1): Linear (100 -> 1)
)
embd torch.Size([1, 12, 64])
unsq torch.Size([1, 1, 12, 64])
conv[0](x) torch.Size([1, 100, 12, 60])
x[0] torch.Size([1, 100, 720])
after maxpool torch.Size([1, 100])
out torch.Size([1, 100])


In [56]:
x = torch.Tensor([1, 2, 3, 4])
print(x.size())
x.unsqueeze_(0)
print(x.size())

torch.Size([4])
torch.Size([1, 4])
